## case 3. Meta-Learning of Low-Resource Neural Machine Translation

- Multilingual Translation System을 만드는 이유 중 하나는 Low-Resource Language pair에 대해 Translation System을 만들기 위해서 한 것이다.


- 그래서 각 언어마다 Separate encoder/decoders([[Firat et al., 2016a; Firat et al., 2016b]]) 를 사용했던 것을 Universal encoder/decoders로([Johnson et al., 2016; Ha et al., 2016; Lee et al., 2017; Gu et al., 2018]) 하면은 
    - case2에서 봤던 character-level로 가서 Vocab 밑 Lexicon 마다 shared 되니까
        - A shared vocabulary of languageagnostic tokens [J, 2016; H, 2016; L, 2017]
        - Universal lexical representation [G, 2018]
    - training은 모든 Language pair data를 쓰니까, 상대적으로 Low-Resource Language pair에 대해서도 되지 않겠냐



- 그래서 본 저번의 결과
<img  src="./image/img_c2-7.PNG" width="50%"> 

- Fin 2 Eng 보면 잘되는 것 같네, 하지만 Fin & Eng 데이터가 상대적으로 low-resouce이긴 하지만 실은 꽤 많은 양이다.
    - 정말 translation pair 800개 정도만 있어도 잘되게 할 수 있는가
    - 이런 extreme한 case를 보고 싶었는데 잘 안된다.
    
    
- Limitations of Multitask Learning 1.
    - overfitting on low-resource pairs, while underfitting on high-resource pairs하게된다.
    - early stopping하기도 어렵다. 그리고 scheduling과 weighting을 잘하면 정말 되긴 되는데, 과학보단 예술에 가까워 피하고 싶다.
    - See [Firat et al., 2016a] and [Lee et al., 2017] for more discussion.
    
    
- Limitations of Multitask Learning 2.
    - 새로운 언어에 대해서도 번역하고 싶으면, 다시 train부터 parameter tuning까지 해야하는 문제점.
    - 그래서 약간의 fine-tuning만 하는 아이디어
    - Transfer Learning [Zoph et al., 2016; Nguyen & Chiang, 2017]
        - Only re-train a subset of parameters on a new language pair.
        - Many possible strategies, but no clear winning strategy.
    - 나쁘지 않은 아이디어인데, 어디를 fine-tuning하냐에 따라 성능이 왔다갔다함
        - CF) QA Sys의 BERT는 self att. only에 Fine tuning으로 한 것으로 앎.
    

- Limitations of Multitask Learning 3.
    - 그리고 이렇게 위처럼 multitask+transfer learning을 하면 도움이 되는지, 왜 잘되는지, 지금까지 해봤더니 잘되는 것 같은데 그게 어느 수준까지되는지.


### Meta-Learning: MAML [Finn et al., 2018]

- UC 버클리에서 Model-agnostic meta-learning이라는 멋진 아이디어가 나왔다.
    - Translation은 아니었지만, Multi-task Learning을 한다고 했을 때
    - 이미 주어진 task들을 잘 푸는 NN을 찾는 것이 아니라
    - 데이터가 적은 새로운 task가 생겼을 때, train을 조금만 해도 확! 잘되도록 하는 것이 목표.
    - 지금 당장의 주어진 문제를 잘 푸는 게 목표가 아니라 나중에 새로운 문제를 어떻게 빨리 잘 풀 수 있는지를 하는 것.
        - 이것을 2단계를 나누어서 하고, 똑같이 SGD를 쓰고 똑같이 DAG를 할 것인데, 다만 mini-batch 하나만 잡아서 gradient 계산할 때, loss function이 조금 복잡해진다.
    

- ** 1) Simulated learning Step **
    - Given a subset $ {D}_{\mathcal{T}} $ of the training set of task $ \mathcal{T} $, update the model parameters N=1 times.
    - $ \theta^{0} $ : 현재 파라미터에서 새로운 task의 mini-batch 데이터에 대해 SGD해보는 것을 simulate하는 것이다. 근데 실제로 하지는 않고, 만약 gradient descent를 내가 했다면 어떻게 될 것인지 볼것이다.
<img  src="./image/img_c3-1.PNG" width="70%"> 
        

- ** 2) Meta-learning Step **
    - Randomly select a task $ k $ and select a training subset $ D = D_{\mathcal{T}^{k}} $ .
    - Randomly select a validation $ D' = D'_{\mathcal{T}^{k}} $ subset for evaluation.
        - 여기서의 loss가 낮기를 바란다.
    - Update the meta-parameter $ \theta_{0} $  by gradient descent:
        - $ \theta_{0} \gets \theta_{0} + \eta_{0} \nabla_{\theta} \mathcal{ L}^{D'}(\theta{'}) $
        - where $ \nabla_{\theta} \mathcal{ L}^{D'}(\theta{'}) = \nabla_{\theta '} \mathcal{ L}^{D'}(\theta{'}) \nabla_{\theta} (\theta -  \eta \nabla_{\theta} \mathcal{ L}^{D}(\theta)  = \nabla_{\theta '} \mathcal{ L}^{D'}(\theta{'}) - \eta \nabla_{\theta '}  \mathcal{ L}^{D'}(\theta{'}) H_{\theta} ( \mathcal{ L}^{D}(\theta)) $
    - $ (\theta -  \eta \nabla_{\theta} \mathcal{ L}^{D}(\theta)  $ 이부분이 한번 스텝을 했다면
    - $ \nabla_{\theta} \mathcal{ L}^{D'}(\theta{'}) $ 이런 파라미터 일 것이라는 것.
    - 그리고 $  \mathcal{ L}^{D'}(\theta{'}) $ 한번 러닝했다고 했을 때의 loss를 낮추는 나의 새 파라미터를 찾겠다는 것이다.
        - 이 때 gradient의 gradient를 계산해야하여 Hessian이다.(2nd order 미분)
      
- Early stopping based on $ \lVert \theta-\theta_{0}  \rVert ^{2} $
- 계산량이 복잡하지만, 할 수는 있고.
- Fine tuning이 확연히 잘된다.
- 그래서 기존의 것과 새로운 task에서도 잘 되게한다.



### Multitask learning vs. Meta-learning

<img  src="./image/img_c3-2.PNG" width="70%"> 

- Multilingual Transfer Leanring은 Fr, Es ,Pt의 언어가 많이 있어 이를 training한 파라미터에서 움직이면 잘 되겠지? 하는 것

- 이에 비해 Meta Learning은 더이상 Fr, Es, Pt에 대한 문제가 아니라, 아예 새로운 문제와 기존의 문제들이 조금씩 주어졌을 때, 최대한 빨리 Adapt할 수 있는 포인트를 찾는 것
    - 따라서 기존의 task의 overfitting이 생기지 않는다.



### Extension to Neural Machine Translation
- Meta Learning을 그냥 적용하면  I/O mismatch between different tasks된다.
    - Vocabulary mismatch among different languages 때문에
    - 이를 해결하기 위해
    -  Universal lexical representation [Gu et al., 2018] 이걸 많이 사용한다.

- Experiments
    - Source tasks: all the languages from Europarl + Russian
        - Bg→En, Cs→En, Da→En, De→En, El→En, Es→En, Et→En, Fr→En, Hu→En, It→En, Lt→En, Nl→En, Pl→En, Pt→En, Sk→En, Sl→En, Sv→En and Ru→En.

    - Target tasks: (simulated) low-resource language pairs
        - Ro→En, Lv→En, Fi→En, Tr→En and Ko→En
        - 한국어는 잘 안되더라.
        - Approximately 16k target tokens (English side): roughly 800 sentence pairs 일 때 잘 된느지 해보았다.
    - Universal lexical representation: obtained from Wikipedia.를 사용.
        - 위키피디아 페이지 translation 안되있는 것들?
        
- Results:
    - 노랑과 빨강 바가 multi-lingual translation를 한 후 finetuning한 결과
    - 보라와 파랑이 meta learning하고 finetuing한 결과
<img  src="./image/img_c3-3.PNG" width="100%"> 

- Using only 800 examples, reaches up to 65% of fullysupervised models in terms of BLEU.(데이터가 많을 때 65%의 성능, 외계인이오면 이런거 사용 ㅎㅎ)
    
<img  src="./image/img_c3-4.PNG" width="50%"> 

- 데이터를 합칠 때 성능이 올라가긴 한데 한국어는 그래도 잘 안되는 것을 볼 수 있다.
- 또 다른 지표를 보면 데이터가 적을 때 많은 도움을 주는 것을 볼 수 있다.

<img  src="./image/img_c3-5.PNG" width="50%"> 

- 메타러닝을 할 때마다 Fast adapt하는게 잘되냐를 보는 것인데,
    - Training을 하면 할수록 finetuning한게(메타러닝 안한게) 어느 순간 떨어진다.
    - 왜냐하면 파라미터가 주어진 lang pair에 대해 overfit하니까
    - 이에 비해 메타러닝은 처음부터 explicit하게 하여 떨어지는 것 없이 잘 된다.
    - 그래도 차이는 막 크진 않다;;

### Conclusion

- 결론: 메타러닝이 좋고 해볼만 한 것 같다. 
- 그리고 이걸보고 다른 논문을 보면, 지금까지 하던 것을 1st order learning이라고 한다면
    - 점점 high order(새로운 문제에도 잘 풀 수 있는가)
    - 그래도 I/O mismatch과 같은 문제는 있을 수 있다.
    
- Gradual shift toward higher-order learning
    - Learning to optimize [Andrychowicz et al., 2017; and others]
    - Multi-agent modelling (theory of mind) [Foerster et al., 2018 LOLA; and others]
    - Neural architecture search [Zoph & Le, 2016; and others]
    - Hyperparameter search [Luketina et al., 2016; and others]
    - And more on the horizon…


### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 


### Papers:

- Chelsea Finn, et al. "Probabilistic Model-Agnostic Meta-Learning", 2018
    - https://arxiv.org/abs/1806.02817
- Jiatao Gu, et al. "Meta-Learning for Low-Resource Neural Machine Translation", 2018
    - https://arxiv.org/abs/1808.08437
- more at: https://www.edwith.org/deepnlp/lecture/29226/
